Este projeto o trabalho de fim de curso do aluno Igor Peterossi Lopes (RA:174929) para o curso de engenharia elétrica da Unicamp, feito no 2 semestre de 2022.

Este trabalho pretende resolver um problema de classificação binária através de três estratégias distintas: com um classificador implementado com regressão logística, com uma máquina de vetores suporte e com redes neurais. Para que, através dos resultados obtidos, possamos comparar cada método de resolução do problema e identificar a sua eficácia.

O problema que esta sendo analisado é o da classificação de websites em sites de phishing e sites seguros (que não são de phishing), os dados para essa classificação fazem parte do acervo de machine learning da University of California, School of Information and Computer Science, disponível no seguinte link: https://archive.ics.uci.edu/ml/datasets/phishing+websites


In [ ]:
# Configurações técnicas gerais para a realização do projeto

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sns.set_theme(style="whitegrid")

Como primeira etapa, obtemos os dados que serão estudados e depois passamos a fazer pequenas adaptações nestes a fim de viabilizar a sua utilização pelos modelos que serão desenvolvidos

In [ ]:
from sklearn.datasets import fetch_openml

X_data, y_data = fetch_openml(data_id=4534, return_X_y=True)
X_data = X_data.astype("int64")
y_data = y_data.astype("int64")

def map_output(value):
    if value == -1:
        return 0
    return value

y_data = y_data.apply(map_output)

Agora realizamos algumas observações em cima dos dados para que possamos compreender melhor as suas características
Começamos unindo o dataset de features com o dos resultados, para facilitar nossas análises

In [ ]:
complete_data = X_data.copy(deep=True)
complete_data["Result"] = y_data
complete_data.head()

Observando como as nossas classes estão distribuídas (sites classificados como phishing x sites normais), notamos uma quantidade ligeiramente maior de casos "positivos", sites identificados como de phishing, em relação aos casos negativos (sites que não são de phishing), ainda sim o dataset é bem equilibrado

In [ ]:
series = complete_data['Result'].value_counts()
total_classes = pd.DataFrame({'Websites': ['Phishing', 'Oficial'], 'Values': [series[1], series[0]]})
sns.barplot(x = 'Values', y = 'Websites', data = total_classes)

Analisamos também a correlação entre cada feature disponível no nosso dataset e a classificação entre phishing e oficial

In [ ]:
result_correlation_df = complete_data.corr()[['Result']].sort_values(by='Result', ascending=False)
plt.figure(figsize=(8, 12))
mask = np.triu(np.ones_like(result_correlation_df, dtype=bool))
heatmap = sns.heatmap(result_correlation_df, vmin=-1, vmax=1, annot=True, cmap='BrBG')

Com base nas informações de correlação acima, podemos simplificar o nosso dataset eliminando todas as features que apresentam correlação mínima como resultado, ou seja, que pouco influênciam na identificação de um site como de phishing

In [ ]:
X_data = X_data.drop(columns=['having_IP_Address', 'Statistical_report', 'DNSRecord', 'URL_Length', 'having_At_Symbol', 'on_mouseover', 'port', 'Links_pointing_to_page', 'Submitting_to_email', 'RightClick', 'popUpWidnow', 'Favicon', 'Iframe', 'Redirect', 'double_slash_redirecting', 'HTTPS_token', 'Abnormal_URL', 'Shortining_Service'])

A nossa estratégia para execução do projeto será de treinar cada modelo utilizando 3 variações do dataset, primeiro utilizando o dataset original otimizado, depois com o dataset "piorado" ao se retirar as duas features mais impactam na classificação dos sites e, por fim, além de retirar estas 2 features, reduzimos o tamanho do dataset para apenas 2000 resultados.
Com os modelos treinados, determinaremos para cada um deles os f1-scores de treinamento e de teste e o overfiting (diferença entre os f1-scores de treinamento e de teste) e salvamos estas informações em uma tabela para realizarmos a comparação entre cada algoritmo utilizado, teremos 3 tabelas, uma para cada versão do dataset. Plotaremos também a matriz de confusão para cada método, como um parâmetro a mais para aprofundar nossa análise

In [ ]:
orig_analysis_data = {}
wor_analysis_data = {}
wor2_analysis_data = {}

Usaremos as biblioteca scikit-learn e tensorflow para auxiliar no desenvolvimento dos modelos, e para a separação dos nossos dados em partes para treinamento e para validação posterior

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=0)

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

def evaluate_algorithm(X_train, y_train, X_test, y_test, model, model_name, analysis_data):

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    f1_train = f1_score(y_train, y_train_pred, average='macro')
    f1_test = f1_score(y_test, y_test_pred, average='macro')

    overfitting = 100 * (1 - f1_test/f1_train)

    model_results = {'F1-Score Train': f1_train, 'F1-Score Test': f1_test, 'Overfitting': overfitting}

    analysis_data[model_name] = (model_results.values())

    return y_test_pred

def evaluate_neural_network(X_train, y_train, X_test, y_test, model, model_name, analysis_data):

    y_train_pred = model.predict(X_train)
    y_train_pred = np.where(y_train_pred > 0.5, 1, 0)

    y_test_pred = model.predict(X_test)
    y_test_pred = np.where(y_test_pred > 0.5, 1, 0)

    f1_train = f1_score(y_train, y_train_pred, average='macro')
    f1_test = f1_score(y_test, y_test_pred, average='macro')

    overfitting = 100 * (1 - f1_test/f1_train)

    model_results = {'F1-Score Train': f1_train, 'F1-Score Test': f1_test, 'Overfitting': overfitting}

    analysis_data[model_name] = (model_results.values())

    return y_test_pred

def create_confusion_matrix(y_test, y_pred, title):
    cf_matrix = confusion_matrix(y_test, y_pred)

    group_names = ['True Neg','False Pos','False Neg','True Pos']
    group_counts = ["{0:0.0f}".format(value) for value in
                    cf_matrix.flatten()]
    group_percentages = ["{0:.2%}".format(value) for value in
                         cf_matrix.flatten()/np.sum(cf_matrix)]
    labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
              zip(group_names,group_counts,group_percentages)]
    labels = np.asarray(labels).reshape(2,2)

    sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues').set(title=title)

Criamos e treinamos cada um dos algoritmos de machine learning que avaliaremos neste projeto, e também determinamos os parâmetros que servirão de base para a comparação e avaliação da qualidade de cada método para resolver o problema

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from keras.models import Sequential
from keras.layers import Dense

# Logistic Regression Model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

# Support Vectors Model
svm = LinearSVC(C=0.0001)
svm.fit(X_train, y_train)

# 1 Neural Network Model
network1 = Sequential()
network1.add(Dense(12, activation='relu', input_shape=(X_data.shape[1],)))
network1.add(Dense(1, activation='sigmoid'))
network1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
network1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=100, verbose=False)

# 2 Neural Network Model
network2 = Sequential()
network2.add(Dense(12, activation='relu', input_shape=(X_data.shape[1],)))
network2.add(Dense(6, activation='relu', input_shape=(X_data.shape[1],)))
network2.add(Dense(1, activation='sigmoid'))
network2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
network2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=100, verbose=False)

In [ ]:
y_test_pred_reg_log = evaluate_algorithm(X_train, y_train, X_test, y_test, logreg, 'Regressão Logística', orig_analysis_data)
y_test_pred_vect = evaluate_algorithm(X_train, y_train, X_test, y_test, svm, 'Vetores Suporte', orig_analysis_data)
y_test_neural_1 = evaluate_neural_network(X_train, y_train, X_test, y_test, network1, 'Rede Neural 1', orig_analysis_data)
y_test_neural_2 = evaluate_neural_network(X_train, y_train, X_test, y_test, network2, 'Rede Neural 2', orig_analysis_data)

Com as estatísticas de cada algoritmo, montamos a tabela que consolida estes dados

In [ ]:
orig_analysis_df = pd.DataFrame.from_dict(orig_analysis_data, orient='index')
orig_analysis_df.columns = ['F1-score Treinamento', 'F1-score Teste', 'Overfitting (%)']
orig_analysis_df

In [ ]:
create_confusion_matrix(y_test, y_test_pred_reg_log, 'Matriz de Confusão - Regressão Logística')

In [ ]:
create_confusion_matrix(y_test, y_test_pred_vect, 'Matriz de Confusão - Máquina de Vetores Suporte')

In [ ]:
create_confusion_matrix(y_test, y_test_neural_1, 'Matriz de Confusão - Rede Neural 1')

In [ ]:
create_confusion_matrix(y_test, y_test_neural_2, 'Matriz de Confusão - Rede Neural 2')

Repetiremos este mesmo processo acima, agora com o nosso dataset piorado ao se eliminar as 2 melhores features para a classificação

In [ ]:
X_data_edit = X_data.copy(deep=True)
y_data_edit = y_data.copy(deep=True)

# Drop das melhores features
X_data_edit = X_data_edit.drop(columns=['SSLfinal_State'])
X_data_edit = X_data_edit.drop(columns=['URL_of_Anchor'])

X_train_edit, X_test_edit, y_train_edit, y_test_edit = train_test_split(X_data_edit, y_data_edit, test_size=0.3, random_state=0)

In [ ]:
# Logistic Regression Model
logreg.fit(X_train_edit, y_train_edit)

# Support Vectors Model
svm.fit(X_train_edit, y_train_edit)

# 1 Neural Network Model
network1_edit = Sequential()
network1_edit.add(Dense(10, activation='relu', input_shape=(X_data_edit.shape[1],)))
network1_edit.add(Dense(1, activation='sigmoid'))
network1_edit.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
network1_edit.fit(X_train_edit, y_train_edit, validation_data=(X_test_edit, y_test_edit), epochs=10, batch_size=100, verbose=False)

# 2 Neural Network Model
network2_edit = Sequential()
network2_edit.add(Dense(10, activation='relu', input_shape=(X_data_edit.shape[1],)))
network2_edit.add(Dense(5, activation='relu', input_shape=(X_data_edit.shape[1],)))
network2_edit.add(Dense(1, activation='sigmoid'))
network2_edit.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
network2_edit.fit(X_train_edit, y_train_edit, validation_data=(X_test_edit, y_test_edit), epochs=10, batch_size=100, verbose=False)

In [ ]:
y_test_pred_reg_log_edit = evaluate_algorithm(X_train_edit, y_train_edit, X_test_edit, y_test_edit, logreg, 'Regressão Logística', wor_analysis_data)
y_test_pred_vect_edit = evaluate_algorithm(X_train_edit, y_train_edit, X_test_edit, y_test_edit, svm, 'Vetores Suporte', wor_analysis_data)
y_test_neural_1_edit = evaluate_neural_network(X_train_edit, y_train_edit, X_test_edit, y_test_edit, network1_edit, 'Rede Neural 1', wor_analysis_data)
y_test_neural_2_edit = evaluate_neural_network(X_train_edit, y_train_edit, X_test_edit, y_test_edit, network2_edit, 'Rede Neural 2', wor_analysis_data)

In [ ]:
wor_analysis_df = pd.DataFrame.from_dict(wor_analysis_data, orient='index')
wor_analysis_df.columns = ['F1-score Treinamento', 'F1-score Teste', 'Overfitting (%)']
wor_analysis_df

In [ ]:
create_confusion_matrix(y_test_edit, y_test_pred_reg_log_edit, 'Matriz de Confusão - Regressão Logística')

In [ ]:
create_confusion_matrix(y_test_edit, y_test_pred_vect_edit, 'Matriz de Confusão - Máquina de Vetores Suporte')

In [ ]:
create_confusion_matrix(y_test_edit, y_test_neural_1_edit, 'Matriz de Confusão - Rede Neural 1')

In [ ]:
create_confusion_matrix(y_test_edit, y_test_neural_2_edit, 'Matriz de Confusão - Rede Neural 2')

E por fim, faremos o processo com o dataset piorado ao se eliminar as 2 melhores features para a classificação e também reduzindo o seu tamanho para 1000 linhas

In [ ]:
X_data_edit_2 = X_data.copy(deep=True)
y_data_edit_2 = y_data.copy(deep=True)

y_data_edit_2 = y_data_edit_2[:2000]
X_data_edit_2 = X_data_edit_2[:2000]

# Drop das melhores features
X_data_edit_2 = X_data_edit_2.drop(columns=['SSLfinal_State'])
X_data_edit_2 = X_data_edit_2.drop(columns=['URL_of_Anchor'])

X_train_edit_2, X_test_edit_2, y_train_edit_2, y_test_edit_2 = train_test_split(X_data_edit_2, y_data_edit_2, test_size=0.3, random_state=0)

In [ ]:
# Logistic Regression Model
logreg.fit(X_train_edit_2, y_train_edit_2)

# Support Vectors Model
svm.fit(X_train_edit_2, y_train_edit_2)

# 1 Neural Network Model
network1_edit_2 = Sequential()
network1_edit_2.add(Dense(10, activation='relu', input_shape=(X_data_edit_2.shape[1],)))
network1_edit_2.add(Dense(1, activation='sigmoid'))
network1_edit_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
network1_edit_2.fit(X_train_edit_2, y_train_edit_2, validation_data=(X_test_edit_2, y_test_edit_2), epochs=10, batch_size=100,verbose=False)

# 2 Neural Network Model
network2_edit_2 = Sequential()
network2_edit_2.add(Dense(10, activation='relu', input_shape=(X_data_edit_2.shape[1],)))
network2_edit_2.add(Dense(5, activation='relu', input_shape=(X_data_edit_2.shape[1],)))
network2_edit_2.add(Dense(1, activation='sigmoid'))
network2_edit_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
network2_edit_2.fit(X_train_edit_2, y_train_edit_2, validation_data=(X_test_edit_2, y_test_edit_2), epochs=10, batch_size=100, verbose=False)

In [ ]:
y_test_pred_reg_log_edit_2 = evaluate_algorithm(X_train_edit_2, y_train_edit_2, X_test_edit_2, y_test_edit_2, logreg, 'Regressão Logística', wor2_analysis_data)
y_test_pred_vect_edit_2 = evaluate_algorithm(X_train_edit_2, y_train_edit_2, X_test_edit_2, y_test_edit_2, svm, 'Vetores Suporte', wor2_analysis_data)
y_test_neural_1_edit_2 = evaluate_neural_network(X_train_edit_2, y_train_edit_2, X_test_edit_2, y_test_edit_2, network1_edit_2, 'Rede Neural 1', wor2_analysis_data)
y_test_neural_2_edit_2 = evaluate_neural_network(X_train_edit_2, y_train_edit_2, X_test_edit_2, y_test_edit_2, network2_edit_2, 'Rede Neural 2', wor2_analysis_data)

In [ ]:
wor2_analysis_df = pd.DataFrame.from_dict(wor2_analysis_data, orient='index')
wor2_analysis_df.columns = ['F1-score Treinamento', 'F1-score Teste', 'Overfitting (%)']
wor2_analysis_df

In [ ]:
create_confusion_matrix(y_test_edit_2, y_test_pred_reg_log_edit_2, 'Matriz de Confusão - Regressão Logística')

In [ ]:
create_confusion_matrix(y_test_edit_2, y_test_pred_vect_edit_2, 'Matriz de Confusão - Máquina de Vetores Suporte')

In [ ]:
create_confusion_matrix(y_test_edit_2, y_test_neural_1_edit_2, 'Matriz de Confusão - Rede Neural 1')

In [ ]:
create_confusion_matrix(y_test_edit_2, y_test_neural_2_edit_2, 'Matriz de Confusão - Rede Neural 2')